In [1]:
import io
from copy import copy
from collections import OrderedDict
#from hamcrest import none
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import h5py
import mne
import hypyp
import requests
import os
import PyQt5
import sys
import copy
import pickle
import statsmodels.api as sm
import autoreject
from statsmodels.formula.api import ols
#print(sys.path)
sys.path.append('C:/Users/Administrateur/MilitaryCoordination/')

from hypyp import (
    prep,)  # need pip install https://api.github.com/repos/autoreject/autoreject/zipball/master
from hypyp import analyses
from hypyp import prep
from hypyp import stats
from hypyp import viz

In [2]:
biosemi64_montage = mne.channels.make_standard_montage('biosemi64')
# change the channel names in our epochs so that they are the same as the montage
channels = biosemi64_montage.ch_names
freq_bands = {'Theta': [4, 7],
            'Alpha': [8, 12],
            'Beta': [13, 30],
            'Gamma': [30, 45]}

freq_list = ['Theta', 'Alpha', 'Beta', 'Gamma']

ROI_channel_names = dict()
ROI_channel_names["left_frontal"] = ["Fp1", "AF7", "AF3", "F7", "F5", "F3"]
ROI_channel_names["right_frontal"] = ["Fp2", "AF8", "AF4", "F8", "F6", "F4"]

ROI_channel_names["left_central"] = ["FC1", "FC3", "C3", "C1", "CP3","CP1"]
ROI_channel_names["right_central"] = ["FC2", "FC4", "C4", "C2", "CP2","CP4"]

ROI_channel_names["left_temporal"] = ["T7", "TP7", "CP5", "P9", "P7", "P5"]
ROI_channel_names["right_temporal"] = ["T8", "TP8", "CP6", "P10", "P8", "P6"]

ROI_channel_numbers = copy.deepcopy(ROI_channel_names)

for key in ROI_channel_names.keys():
    for i in range(len(ROI_channel_names[key])):
        ch_name = ROI_channel_names[key][i]
        index = channels.index(ch_name)
        ROI_channel_numbers[key][i] = index

ROI_keys = list(ROI_channel_numbers.keys())
print(ROI_keys)

['left_frontal', 'right_frontal', 'left_central', 'right_central', 'left_temporal', 'right_temporal']


In [4]:
import pandas as pd
connectivity_df = pd.DataFrame(columns = ["pair", "group", "condition", "frequency", "ROI_1", "ROI_2", "ROI_combination", "wPLI"])
event_id = {'Synchronous/Egalitarian': 2, 'Synchronous/LeaderFollower': 3, 'Synchronous/FollowerLeader': 4, 'Individual': 5, 'Complementary/Egalitarian': 6, 'Complementary/LeaderFollower': 7, 'Complementary/FollowerLeader': 8}
n_ch = 64
i_df = 0
for pair in range(1,44):

    if (pair) > 20:
        group = 'civilian'
    else:
        group = 'military'

    ######### load in files ##############
    
    try:
        file_path = os.path.join("F:/hyperscanning_mne/", "time_locked", "plv_time_locked_connectivity_values_pair_" + str(pair))
        #file_path = os.path.join("F:/hyperscanning_mne", "connectivity_values_pair_" + str(pair))
        print(file_path)
        with open(file_path , "rb") as input_file:
            connectivity_values = pickle.load(input_file) 

        file_path = os.path.join("F:/hyperscanning_mne/", "time_locked","pair_" + str(pair))     
        with open(file_path , "rb") as input_file:
            cleaned_epochs_AR, dic_AR = pickle.load(input_file)

        file_path = os.path.join("F:/hyperscanning_mne/","time_locked", "bad_labels_pair_" + str(pair))
        with open(file_path , "rb") as input_file:
            bad_labels = pickle.load(input_file) 
                
    except:
        print('pair not available')
        continue
    


    preproc_S1 = cleaned_epochs_AR[0]
    preproc_S2 = cleaned_epochs_AR[1]


    ###### get positions of conditions for extra rejection steps##############
    event_id = {'Synchronous/Egalitarian': 2, 'Synchronous/LeaderFollower': 3, 'Synchronous/FollowerLeader': 4, 'Individual': 5, 'Complementary/Egalitarian': 6, 'Complementary/LeaderFollower': 7, 'Complementary/FollowerLeader': 8}
    epoch_position_dict = {}

    print('calculate complex signal for all conditions')
    for condition in event_id.keys():

        # find the epoch positions of this condition
        epoch_positions = np.where(preproc_S1.events[:,2] == event_id[condition])

        if event_id[condition] in preproc_S1.events[:,2]:
            epoch_position_dict[condition] = epoch_positions



    ########### apply the extra rejection step to the data ###################
    for condition in connectivity_values.keys():

        try: 
            results = connectivity_values[condition]
        except:
            continue
            
        print(pair)
        print(condition)
        print(epoch_position_dict)
        print(np.shape(bad_labels[0]))

        # get part of the rejection_log that corresponds to condition
        condition_log_1 = bad_labels[0][epoch_position_dict[condition]]
        condition_log_2 = bad_labels[1][epoch_position_dict[condition]]
        
        
        # set the correct positions of the connectivity matrix to nans
        for ch_1 in range(64):
            # intra-brain
            bad_epochs = np.where((condition_log_1[:,ch_1] == 1))
            results[:,bad_epochs, :, ch_1, ch_1] = np.nan

            for ch_2 in range(64):
                # intra-brain for channel 2
                bad_epochs = np.where((condition_log_2[:,ch_2] == 1))
                results[:,bad_epochs, :, n_ch+ch_2, n_ch+ch_2] = np.nan

                # find bad segments for inter-brain
                bad_epochs = np.clip(condition_log_1[:,ch_1] + condition_log_2[:,ch_2], 0, 1)
                bad_epochs = bad_epochs.astype(int)
                results[:,bad_epochs, :, ch_1, n_ch+ch_2] = np.nan
                results[:,bad_epochs, :, n_ch+ch_2, ch_1] = np.nan

        connectivity_values[condition] = results



    ############### average the connectivity values into ROIs #################################
    for condition in connectivity_values.keys():
        condition_data = connectivity_values[condition]
        print(condition_data.shape)
        # first get the inter_brain part
        n_ch = 64
        condition_data = condition_data[:, :, :, 0:n_ch, n_ch:2*n_ch]

        # loop over all frequencies and inter-brain pairs
        for frequency in range(4):
            freq_string = freq_list[frequency]

            # loop over all ROI's
            for ROI_1 in range(6):
                ROI_1_string = ROI_keys[ROI_1]
                channel_numbers_1 = ROI_channel_numbers[ROI_1_string]
                for ROI_2 in range(6):
                    ROI_2_string = ROI_keys[ROI_2]
                    ROI_combination = ROI_1_string + "-" + ROI_2_string

                    channel_numbers_2 = ROI_channel_numbers[ROI_2_string]
                    

                    # first average within the epochs and then between the epochs
                    n_subs = np.size(condition_data, 2)
                    epoch_list = np.empty((0,n_subs))
                    for epoch in range(np.shape(condition_data)[1]):
                        epoch_value = 0
                        counter = 0
                        for ch_1 in channel_numbers_1:
                            for ch_2 in channel_numbers_2:
                                if not np.isnan(condition_data[frequency, epoch, :, ch_1, ch_2]):
                                    epoch_value+=condition_data[frequency, epoch, :,  ch_1, ch_2]
                                    counter+=1
                        if counter < 9: # if there are less than 9 connections (3x3) in the ROI combination, the epoch should not be taken into account
                            continue

                        try:
                            epoch_list = np.vstack((epoch_list, epoch_value))
                        except:
                            # if all segments are bad then skip the epoch
                            continue

                    ROI_connectivity = np.nanmean(epoch_list, axis = 0)

                    plt.plot(ROI_connectivity)
                    plt.show()

                    plt.plot(ROI_connectivity - ROI_connectivity[0])
                    plt.show()




                    datapoint = pd.Series(data=[pair, group, condition, freq_string, ROI_1_string, ROI_2_string, ROI_combination, ROI_connectivity],  index= connectivity_df.columns, name = i_df)
                                
                    connectivity_df = pd.concat([connectivity_df, datapoint.to_frame().T], ignore_index = True) #grid_results.append(configuration_result)
                    i_df+=1
print(connectivity_df)


F:/hyperscanning_mne/time_locked\plv_time_locked_connectivity_values_pair_1
calculate complex signal for all conditions
1
Synchronous/Egalitarian
{'Synchronous/Egalitarian': (array([138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
       151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,
       164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176,
       177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189,
       190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202,
       203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,
       216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228,
       229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241,
       242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254,
       255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267,
       268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 2

IndexError: index 64 is out of bounds for axis 2 with size 11